As the third step of this tutorial, we will train an text model. This notebook can be run in parallel with Step 2 (training the image model). A lof of the cells in this notebook are similar to the previous one.

This notebook was run on an AWS p3.2xlarge.

# Octopod Text Model Training Pipeline

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, BertTokenizer, get_cosine_schedule_with_warmup

Note: for text, we use the MultiTaskLearner since we will only have one input, the text.

In [4]:
from octopod import MultiTaskLearner, MultiDatasetLoader
from octopod.text.dataset import OctopodTextDataset
from octopod.text.models.multi_task_bert import BertForMultiTaskClassification

For our Bert model, we need a tokenizer. We'll use the one from huggingface's `transformers` library.

In [5]:
bert_tok = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [6]:
TRAIN_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_train.csv')

In [8]:
VALID_GENDER_DF = pd.read_csv('/home/ubuntu/fashion_dataset/gender_valid.csv')

In [9]:
TRAIN_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_train.csv')

In [10]:
VALID_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [11]:
batch_size = 64

We use the `OctopodTextDataSet` class to create train and valid datasets for each task.

Check out the documentation for infomation about the `tokenizer` and `max_seq_length` arguments.

In [12]:
max_seq_length = 128

In [13]:
gender_train_dataset = OctopodTextDataset(
    x=TRAIN_GENDER_DF['productDisplayName'],
    y=TRAIN_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length
)
gender_valid_dataset = OctopodTextDataset(
    x=VALID_GENDER_DF['productDisplayName'],
    y=VALID_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length
)

season_train_dataset = OctopodTextDataset(
    x=TRAIN_SEASON_DF['productDisplayName'],
    y=TRAIN_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length
)
season_valid_dataset = OctopodTextDataset(
    x=VALID_SEASON_DF['productDisplayName'],
    y=VALID_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length
)

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [14]:
train_dataloaders_dict = {
    'gender': DataLoader(gender_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'season': DataLoader(season_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
}
valid_dataloaders_dict = {
    'gender': DataLoader(gender_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
    'season': DataLoader(season_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
}

The dictionary of dataloaders is then put into an instance of the Octopod `MultiDatasetLoader` class.

In [15]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

730

In [16]:
ValidLoader = MultiDatasetLoader(loader_dict=valid_dataloaders_dict, shuffle=False)
len(ValidLoader)

244

We need to create a dictionary of the tasks and the number of unique values so that we can create our model.

In [17]:
new_task_dict = {
    'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
    'season': TRAIN_SEASON_DF['season_cat'].nunique(),
}

In [18]:
new_task_dict

{'gender': 5, 'season': 4}

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Create Model and Learner
===

These are completely new tasks so we use `new_task_dict`. If we had already trained a model on some tasks, we would use `pretrained_task_dict`.

We are using the trained bert weights from the `transformers` library.

In [20]:
model = BertForMultiTaskClassification.from_pretrained(
    'bert-base-uncased',
    new_task_dict=new_task_dict
)

You will likely need to explore different values in this section to find some that work
for your particular model.

In [21]:
loss_function = nn.CrossEntropyLoss()

lr = 1e-5
num_total_steps = len(TrainLoader)
num_warmup_steps = int(len(TrainLoader) * 0.1)

optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_total_steps
)

In [22]:
learn = MultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict)

Train Model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [23]:
learn.fit(
    num_epochs=10,
    loss_function=loss_function,
    scheduler=scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,gender_train_loss,gender_val_loss,gender_acc,season_train_loss,season_val_loss,season_acc,time
1.443156,1.436217,1.496199,1.494172,0.420691,1.372393,1.358906,0.188317,06:26
1.262111,1.149637,1.245156,1.084975,0.495216,1.284731,1.235895,0.483706,06:29
1.017829,0.889183,0.865197,0.652203,0.849263,1.221454,1.205311,0.483706,06:29
0.782725,0.708237,0.472487,0.352095,0.906788,1.196609,1.183323,0.483706,06:27
0.674205,0.647238,0.297471,0.255677,0.946077,1.176801,1.169574,0.483706,06:28
0.615526,0.594338,0.204330,0.173638,0.953282,1.164096,1.155545,0.483706,06:28
0.572668,0.555147,0.141206,0.117855,0.980525,1.148274,1.138487,0.483706,06:28
0.545199,0.528069,0.107176,0.091960,0.982664,1.129557,1.109831,0.511188,06:27
0.509614,0.505068,0.088725,0.081082,0.984465,1.071116,1.070659,0.524103,06:29
0.464816,0.494566,0.075624,0.072447,0.987279,0.984032,1.057667,0.525304,06:28


Epoch 9 best model saved with loss of 0.49456640841227717


Validate Model
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [24]:
pred_dict = learn.get_val_preds(device)

In [25]:
pred_dict

{'gender': {'y_true': array([0., 2., 4., ..., 4., 2., 2.]),
  'y_pred': array([[0.74698263, 0.10318641, 0.08654781, 0.03519604, 0.02808712],
         [0.00239562, 0.00217793, 0.99014109, 0.00213638, 0.00314901],
         [0.00328295, 0.00526687, 0.00393449, 0.00815226, 0.97936338],
         ...,
         [0.00183818, 0.00299671, 0.00261172, 0.00342717, 0.98912621],
         [0.01128249, 0.00917029, 0.95130181, 0.01607285, 0.01217249],
         [0.00264054, 0.00205473, 0.98852623, 0.00422364, 0.00255485]])},
 'season': {'y_true': array([0., 0., 2., ..., 2., 3., 3.]),
  'y_pred': array([[0.20209727, 0.06460249, 0.37908569, 0.35421461],
         [0.2017833 , 0.06096645, 0.42957458, 0.30767566],
         [0.1913355 , 0.06862341, 0.3871206 , 0.35292041],
         ...,
         [0.16365369, 0.06352966, 0.36214185, 0.41067481],
         [0.10442591, 0.05663429, 0.08722814, 0.75171155],
         [0.05810045, 0.06206312, 0.0734309 , 0.80640548]])}}

Save/Export Model
===

Once we are happy with our training we can save (or export) our model, using the `save` method (or `export`).

See the docs for the difference between `save` and `export`.

We will need the saved model later to use in the ensemble model

In [26]:
model.save(folder='/home/ubuntu/fashion_dataset/models/', model_id='TEXT_MODEL1')

In [27]:
model.export(folder='/home/ubuntu/fashion_dataset/models/', model_id='TEXT_MODEL1')

Now that we have an image model and a text model, we can move to `Step4_train_ensemble_model`.